In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
import undetected_chromedriver as uc

In [2]:
def initDriver(url):
    options = Options()
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = uc.Chrome(options=options)
    driver.get(url)
    return driver

In [3]:
def closeWindow(driver):
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "close")))
    driver.find_element(By.CLASS_NAME, "close").click()
    time.sleep(1)

In [4]:
def scrapeLeague(driver, league, projectionLst):
    time.sleep(1)
    categories = driver.find_element(By.CSS_SELECTOR, ".stat-container").text.split('\n')

    for category in categories:
        driver.find_element(By.XPATH, f"//div[text()='{category}']").click()
        projections = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".projection")))

        for projection in projections:
            names = projection.find_element(By.XPATH, './/div[@class="name"]').text
            points = projection.find_element(By.XPATH, './/div[@class="presale-score"]').get_attribute('innerHTML')
            text = projection.find_element(By.XPATH, './/div[@class="text"]').text.replace('\n','')
            team = projection.find_element(By.XPATH, './/div[@class="team-position"]').text.split(' - ')[0]
            position = projection.find_element(By.XPATH, './/div[@class="team-position"]').text.split(' - ')[1]
            date = projection.find_element(By.XPATH, './/div[@class="date"]').text
            opponent = projection.find_element(By.XPATH, './/div[@class="opponent"]').text.replace('vs ','')
            players = {'Name': names, 'Prop': points, 'Line': text, 'League': league.text, 'Team': team, 'Position': position, 'Date': date, 'Opponent': opponent, 'ScrapeDate': datetime.now().replace(microsecond=0)}
            projectionLst.append(players)
            
    return projectionLst

In [5]:
def scrapeProjs(driver):
    stat_container = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "stat-container")))
    leagues = driver.find_elements(By.CLASS_NAME, "league")
    projectionLst = []

    for league in leagues:
        league.click()
        projectionLst = scrapeLeague(driver, league, projectionLst)
        time.sleep(1)

    return pd.DataFrame(projectionLst)

In [6]:
def run():
    url = "https://app.prizepicks.com/"
    driver = initDriver(url)
    closeWindow(driver)
    projs = pd.DataFrame()
    
    try:
        projs = scrapeProjs(driver)
    finally:
        driver.quit()
    
    return projs

In [7]:
df = run()

In [8]:
dfCopy = df.copy()

In [9]:
df = dfCopy.copy()

In [10]:
df.shape

(628, 9)

In [11]:
df.head()

,Name,Prop,Line,League,Team,Position,Date,Opponent,ScrapeDate
0,Jimmy Butler,26.5,Points,NBA,MIA,F,"Mon, Jun 12 5:40 PM",DEN,2023-06-11 15:21:26
1,Nikola Jokic,29.5,Points,NBA,DEN,C,"Mon, Jun 12 5:40 PM",MIA,2023-06-11 15:21:26
2,Bam Adebayo,20.5,Points,NBA,MIA,C-F,"Mon, Jun 12 5:40 PM",DEN,2023-06-11 15:21:27
3,Jamal Murray,25.5,Points,NBA,DEN,G,"Mon, Jun 12 5:40 PM",MIA,2023-06-11 15:21:27
4,Aaron Gordon,13.5,Points,NBA,DEN,F,"Mon, Jun 12 5:40 PM",MIA,2023-06-11 15:21:27


In [12]:
df.tail()

,Name,Prop,Line,League,Team,Position,Date,Opponent,ScrapeDate
623,Jaxon Smith-Njigba,725.5,Receiving Yards,NFLSZN,SEA,WR,"Thu, Sep 07 5:15 PM",2023 SEA,2023-06-11 15:22:32
624,Quentin Johnston,555.5,Receiving Yards,NFLSZN,LAC,WR,"Thu, Sep 07 5:15 PM",2023 LAC,2023-06-11 15:22:32
625,Zay Flowers,625.5,Receiving Yards,NFLSZN,BAL,WR,"Thu, Sep 07 5:15 PM",2023 BAL,2023-06-11 15:22:32
626,Jordan Addison,825.5,Receiving Yards,NFLSZN,MIN,WR,"Thu, Sep 07 5:15 PM",2023 MIN,2023-06-11 15:22:32
627,Odell Beckham Jr.,575.5,Receiving Yards,NFLSZN,BAL,WR,"Thu, Sep 07 5:15 PM",2023 BAL,2023-06-11 15:22:32


In [13]:
def convertMonth(dateStr):
    months = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}
    a = dateStr.strip()[5:8].lower()
    return months[a]

In [14]:
def convertDay(dateStr):
    return dateStr[9:11]

In [15]:
def convertTime(dateStr):
    t = datetime.strptime(dateStr[12:], '%I:%M %p')
    return t.strftime('%H:%M:%S')

In [16]:
df['Date'] = [datetime.strptime('23-' + convertMonth(x) + '-' + str(convertDay(x)) + ' ' + str(convertTime(x)), '%y-%m-%d %H:%M:%S').replace(second=0) if x[0:5] != 'Start' else x for x in df.Date]

In [17]:
for index, row in df.iterrows():
    if isinstance(row['Date'], str):
        newTime = row['ScrapeDate']
        newTime += timedelta(minutes=int(row['Date'][7:9]), seconds=int(row['Date'][10:12]))
        df.at[index, 'Date'] = newTime.replace(second=0)

In [18]:
df.tail()

,Name,Prop,Line,League,Team,Position,Date,Opponent,ScrapeDate
623,Jaxon Smith-Njigba,725.5,Receiving Yards,NFLSZN,SEA,WR,2023-09-07 17:15:00,2023 SEA,2023-06-11 15:22:32
624,Quentin Johnston,555.5,Receiving Yards,NFLSZN,LAC,WR,2023-09-07 17:15:00,2023 LAC,2023-06-11 15:22:32
625,Zay Flowers,625.5,Receiving Yards,NFLSZN,BAL,WR,2023-09-07 17:15:00,2023 BAL,2023-06-11 15:22:32
626,Jordan Addison,825.5,Receiving Yards,NFLSZN,MIN,WR,2023-09-07 17:15:00,2023 MIN,2023-06-11 15:22:32
627,Odell Beckham Jr.,575.5,Receiving Yards,NFLSZN,BAL,WR,2023-09-07 17:15:00,2023 BAL,2023-06-11 15:22:32


In [19]:
df.head()

,Name,Prop,Line,League,Team,Position,Date,Opponent,ScrapeDate
0,Jimmy Butler,26.5,Points,NBA,MIA,F,2023-06-12 17:40:00,DEN,2023-06-11 15:21:26
1,Nikola Jokic,29.5,Points,NBA,DEN,C,2023-06-12 17:40:00,MIA,2023-06-11 15:21:26
2,Bam Adebayo,20.5,Points,NBA,MIA,C-F,2023-06-12 17:40:00,DEN,2023-06-11 15:21:27
3,Jamal Murray,25.5,Points,NBA,DEN,G,2023-06-12 17:40:00,MIA,2023-06-11 15:21:27
4,Aaron Gordon,13.5,Points,NBA,DEN,F,2023-06-12 17:40:00,MIA,2023-06-11 15:21:27
